<a href="https://colab.research.google.com/github/HudsonDev-Hub/Pizzaria_POO/blob/main/Projeto_pizzaria_Hudson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Um sistema de pedidos de pizzas desenvolvido em Python, aplicando os principais conceitos de Programação Orientada a Objetos (POO), como abstração, herança, encapsulamento e persistência de dados com SQLite.

import sqlite3
import datetime
from abc import ABC, abstractmethod

#ABSTRAÇÃO
class Pessoa(ABC):
    def __init__(self, nome, email, telefone):
        self._nome = nome
        self._email = email
        self._telefone = telefone

    @abstractmethod
    def exibir_dados(self):
        pass

#HERANÇA + ENCAPSULAMENTO
class Cliente(Pessoa):
    def __init__(self, nome, email, telefone):
        super().__init__(nome, email, telefone)

    def exibir_dados(self):
        print(f"👤 Cliente: {self._nome} | Email: {self._email} | Telefone: {self._telefone}")

#CLASSE PIZZA
class Pizza:
    def __init__(self, sabor, preco):
        self._sabor = sabor
        self._preco = preco

    def __str__(self):
        return f"{self._sabor} - R${self._preco:.2f}"

#CLASSE PEDIDO
class Pedido:
    def __init__(self, cliente_id, endereco):
        self._cliente_id = cliente_id
        self._data = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self._itens = []
        self._total = 0.0
        self._endereco = endereco

    def adicionar_item(self, pizza_id, sabor, preco, quantidade):
        self._itens.append((pizza_id, sabor, preco, quantidade))
        self._total += preco * quantidade

    def calcular_total_com_frete(self):
        frete = 8.0 if self.total_pizzas() >= 2 else 0.0
        return self._total + frete

    def total_pizzas(self):
        return sum(qtd for _, _, _, qtd in self._itens)

    def exibir_resumo(self):
        print("\n🧾 Resumo do Pedido:")
        for _, sabor, preco, qtd in self._itens:
            print(f"  - {qtd}x {sabor} (R${preco:.2f})")
        print(f"🏠 Endereço de entrega: {self._endereco}")
        print(f"📦 Total com frete: R${self.calcular_total_com_frete():.2f}")

#BANCO DE DADOS
class BancoDeDados:
    def __init__(self, nome_bd="pizzaria.db"):
        self._conexao = sqlite3.connect(nome_bd)
        self._cursor = self._conexao.cursor()
        self._criar_tabelas()

    def _criar_tabelas(self):
        self._cursor.execute("""
            CREATE TABLE IF NOT EXISTS clientes (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                nome TEXT NOT NULL,
                email TEXT UNIQUE,
                telefone TEXT
            )
        """)
        self._cursor.execute("""
            CREATE TABLE IF NOT EXISTS pizzas (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                sabor TEXT NOT NULL,
                preco REAL NOT NULL
            )
        """)
        self._cursor.execute("""
            CREATE TABLE IF NOT EXISTS pedidos (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                cliente_id INTEGER,
                data TEXT,
                endereco TEXT,
                FOREIGN KEY(cliente_id) REFERENCES clientes(id)
            )
        """)
        self._cursor.execute("""
            CREATE TABLE IF NOT EXISTS itens_pedido (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                pedido_id INTEGER,
                pizza_id INTEGER,
                quantidade INTEGER,
                FOREIGN KEY(pedido_id) REFERENCES pedidos(id),
                FOREIGN KEY(pizza_id) REFERENCES pizzas(id)
            )
        """)
        self._conexao.commit()

    def cadastrar_cliente(self, cliente):
        self._cursor.execute("SELECT id FROM clientes WHERE email = ?", (cliente._email,))
        cliente_existente = self._cursor.fetchone()
        if cliente_existente:
            print(f"⚠ Cliente já cadastrado: {cliente._nome}")
            return cliente_existente[0]
        self._cursor.execute("INSERT INTO clientes (nome, email, telefone) VALUES (?, ?, ?)",
                             (cliente._nome, cliente._email, cliente._telefone))
        self._conexao.commit()
        print(f"✅ Cliente {cliente._nome} cadastrado com sucesso.")
        return self._cursor.lastrowid

    def popular_cardapio(self):
        self._cursor.execute("SELECT COUNT(*) FROM pizzas")
        if self._cursor.fetchone()[0] == 0:
            pizzas = [
                ("Mussarela", 30.0),
                ("Calabresa", 35.0),
                ("Frango com Catupiry", 38.0),
                ("Portuguesa", 40.0),
            ]
            self._cursor.executemany("INSERT INTO pizzas (sabor, preco) VALUES (?, ?)", pizzas)
            self._conexao.commit()
            print("📋 Cardápio carregado.")

    def mostrar_cardapio(self):
        self._cursor.execute("SELECT * FROM pizzas")
        pizzas = self._cursor.fetchall()
        print("\n🍕 Cardápio de Pizzas:")
        for pid, sabor, preco in pizzas:
            print(f"{pid}. {sabor} - R${preco:.2f}")

    def fazer_pedido(self, cliente_id):
        endereco = input("Informe o endereço de entrega: ")
        pedido = Pedido(cliente_id, endereco)
        self.mostrar_cardapio()

        while True:
            pizza_id = input("Digite o ID da pizza (ou 'fim' para encerrar): ")
            if pizza_id.lower() == 'fim':
                break
            if not pizza_id.isdigit():
                print("⚠ ID inválido.")
                continue
            pizza_id = int(pizza_id)

            self._cursor.execute("SELECT sabor, preco FROM pizzas WHERE id = ?", (pizza_id,))
            pizza_info = self._cursor.fetchone()
            if not pizza_info:
                print("⚠ Pizza não encontrada.")
                continue

            try:
                quantidade = int(input("Quantidade: "))
                if quantidade <= 0:
                    print("⚠ Quantidade deve ser maior que zero.")
                    continue
                sabor, preco = pizza_info
                pedido.adicionar_item(pizza_id, sabor, preco, quantidade)
                print(f"✅ {quantidade}x {sabor} adicionadas.")
            except ValueError:
                print("⚠ Entrada inválida.")

        if pedido.total_pizzas() == 0:
            print("❌ Pedido vazio.")
            return

        # Salvar pedido no banco
        self._cursor.execute(
            "INSERT INTO pedidos (cliente_id, data, endereco) VALUES (?, ?, ?)",
            (cliente_id, pedido._data, pedido._endereco)
        )
        pedido_id = self._cursor.lastrowid

        for pizza_id, _, _, qtd in pedido._itens:
            self._cursor.execute(
                "INSERT INTO itens_pedido (pedido_id, pizza_id, quantidade) VALUES (?, ?, ?)",
                (pedido_id, pizza_id, qtd)
            )

        self._conexao.commit()
        pedido.exibir_resumo()

        print("\n✅ Pedido registrado com sucesso! Obrigado por comprar conosco.")
        self.fechar()
        exit()

    def listar_pedidos(self):
        self._cursor.execute("""
            SELECT pedidos.id, clientes.nome, pedidos.data, pedidos.endereco
            FROM pedidos
            JOIN clientes ON pedidos.cliente_id = clientes.id
            ORDER BY pedidos.data DESC
        """)
        pedidos = self._cursor.fetchall()
        print("\n📦 Pedidos Realizados:")
        for pid, nome, data, endereco in pedidos:
            print(f"\nPedido #{pid} - Cliente: {nome} - Data: {data}")
            print(f"🏠 Endereço: {endereco}")
            self._cursor.execute("""
                SELECT pizzas.sabor, itens_pedido.quantidade
                FROM itens_pedido
                JOIN pizzas ON itens_pedido.pizza_id = pizzas.id
                WHERE itens_pedido.pedido_id = ?
            """, (pid,))
            itens = self._cursor.fetchall()
            for sabor, qtd in itens:
                print(f"  - {qtd}x {sabor}")

    def fechar(self):
        print("\n🔒 Encerrando conexão com o banco de dados.")
        self._conexao.close()

# PROGRAMA PRINCIPAL
def main():
    print("\n🍕 Bem-vindo à Pizzaria The Brothers 🍕")
    bd = BancoDeDados()
    bd.popular_cardapio()

    nome = input("Informe seu nome: ")
    email = input("Informe seu e-mail: ")
    telefone = input("Informe seu telefone: ")

    cliente = Cliente(nome, email, telefone)
    cliente.exibir_dados()
    cliente_id = bd.cadastrar_cliente(cliente)

    bd.fazer_pedido(cliente_id)

if __name__ == "__main__":
    main()



🍕 Bem-vindo à Pizzaria The Brothers 🍕
